In [16]:
from llama_index import Document, VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.vector_stores import ChromaVectorStore
from llama_index.readers.chroma import ChromaReader
from llama_index.storage.storage_context import StorageContext
# from transformers import AutoTokenizer, AutoModel
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index.schema import MetadataMode
from IPython.display import Markdown, display
import chromadb
import pandas as pd 
import openai
import os
import getpass
import json

In [17]:
!pwd

/rag/jupyter/llama-index


In [45]:
data_path = os.path.join('/rag/data/')
index_path = os.path.join('/rag/db/chroma-llama')
config_path = os.path.join('/rag/config')

In [7]:
query = input("User: ")

User:  재동


In [8]:
query

'재동'

In [19]:
with open(os.path.join(config_path, 'config.json'), "r", encoding="utf-8") as f:
    config = json.load(f)

In [20]:
config['index_path']

'/rag/db/chorma-llama/'

In [21]:
index_path

'/rag/db/chroma-llama'

In [55]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
openai.api_key = os.environ["OPENAI_API_KEY"]

OpenAI API Key: ········


In [46]:
chroma_client = chromadb.PersistentClient(path=index_path)
desc_collection = chroma_client.get_or_create_collection("desc")
feature_collection = chroma_client.get_or_create_collection("feature")
qualification_collection = chroma_client.get_or_create_collection("qualification")

In [47]:
chroma_client.list_collections()

[Collection(name=feature),
 Collection(name=qualification),
 Collection(name=desc)]

In [24]:
list(map(lambda x: x.name, chroma_client.list_collections()))

['feature', 'qualification', 'desc']

In [48]:
qualification_collection.count()   # 2566 

806

In [26]:
desc_store = ChromaVectorStore(chroma_collection=desc_collection)
feature_store = ChromaVectorStore(chroma_collection=feature_collection)
qualification_store = ChromaVectorStore(chroma_collection=qualification_collection)

In [27]:
desc_storage = StorageContext.from_defaults(vector_store=desc_store)
feature_storage = StorageContext.from_defaults(vector_store=feature_store)
qualification_storage = StorageContext.from_defaults(vector_store=qualification_store)

In [28]:
from llama_index import Document, VectorStoreIndex 
from llama_index.node_parser import SentenceSplitter

In [29]:
data = pd.read_csv(os.path.join(data_path, 'financial', 'SHINHAN BANK_Financial_Product_Scraping_Result_20240104_LOAN_vec.csv'))
data.head(1)

,대출_고유키,금융상품명,상품특징,대출신청자격,상품설명
0,대출_0,혁신성장산업 지원자금,9대 선도산업 등 정부발표 혁신성장 분야에 대한 효율적 금융지원,○ (지원대상) 다음 중 하나에 해당하는 기업 혁신성장 정책금융협의회 선정 「혁신성...,9대 선도산업 등 정부발표 혁신성장 분야에 대한 효율적 금융지원


In [30]:
meta_col = ['대출_고유키', '금융상품명']
metadatas = data[meta_col]
list(map(lambda x: x[0], metadatas.values.tolist()))

['대출_0',
 '대출_1',
 '대출_2',
 '대출_3',
 '대출_4',
 '대출_5',
 '대출_6',
 '대출_7',
 '대출_8',
 '대출_9',
 '대출_10',
 '대출_11',
 '대출_12',
 '대출_13',
 '대출_14',
 '대출_15',
 '대출_16',
 '대출_17',
 '대출_18',
 '대출_19',
 '대출_20',
 '대출_21',
 '대출_22',
 '대출_23',
 '대출_24',
 '대출_25',
 '대출_26',
 '대출_27',
 '대출_28',
 '대출_29',
 '대출_30',
 '대출_31',
 '대출_32',
 '대출_33',
 '대출_34',
 '대출_35',
 '대출_36',
 '대출_37',
 '대출_38',
 '대출_39',
 '대출_40',
 '대출_41',
 '대출_42',
 '대출_43',
 '대출_44',
 '대출_45',
 '대출_46',
 '대출_47',
 '대출_48',
 '대출_49',
 '대출_50',
 '대출_51',
 '대출_52',
 '대출_53',
 '대출_54',
 '대출_55',
 '대출_56',
 '대출_57',
 '대출_58',
 '대출_59',
 '대출_60',
 '대출_61',
 '대출_62',
 '대출_63',
 '대출_64',
 '대출_65',
 '대출_66',
 '대출_67',
 '대출_68',
 '대출_69',
 '대출_70',
 '대출_71',
 '대출_72',
 '대출_73',
 '대출_74',
 '대출_75',
 '대출_76',
 '대출_77',
 '대출_78',
 '대출_79',
 '대출_80',
 '대출_81',
 '대출_82',
 '대출_83',
 '대출_84',
 '대출_85',
 '대출_86',
 '대출_87',
 '대출_88',
 '대출_89',
 '대출_90',
 '대출_91',
 '대출_92',
 '대출_93',
 '대출_94',
 '대출_95',
 '대출_96',
 '대출_97',
 '대출_98',
 '대출_99',
 '대출_100',

In [31]:
len(data) - len(data[data.상품특징.isna()])

834

In [32]:
nona = data.copy()

In [33]:
len(data)

1374

In [34]:
data[data.상품특징.isna()].index[:3]

Index([106, 107, 108], dtype='int64')

In [35]:
nona.dropna(subset='상품특징', inplace=True)
len(nona)

834

In [36]:
category = 'loan'

id_list = nona.대출_고유키.values.tolist()
name_list = nona.금융상품명.values.tolist()
text_list = nona.상품특징.values.tolist()

In [37]:
text_list[65]

'소상공인 방역지원금을 지급받는 소기업·소상공인의 비용금융 부담 완화를 위해 한시적으로 대출금리 등 금융지원을 하는 상품개인사업자'

In [38]:
len(id_list), len(text_list)

(834, 834)

In [39]:
documents = []

for idx in range(len(text_list)):
    doc = Document(text=text_list[idx], doc_id=id_list[idx], metadata={"category": category, "name": name_list[idx]}, \
                   excluded_llm_metadata_keys = ['category', 'name'])
    documents.append(doc)

In [252]:
print(f'embedding model see this: {documents[2].get_content(metadata_mode=MetadataMode.EMBED)}', end='\n\n')
print(f'LLM see this: {documents[2].get_content(metadata_mode=MetadataMode.LLM)}')

embedding model see this: category: loan
name: 동산·채권 담보대출 지원자금

동산·채권 담보대출 활성화 지원

LLM see this: 동산·채권 담보대출 활성화 지원


In [60]:
from llama_index.node_parser import SentenceSplitter 

In [61]:
parser = SentenceSplitter(chunk_size=512, chunk_overlap=30)   # SentenceSplitter(chunk_size=1024, chunk_overlap=20)
# nodes = parser.get_nodes_from_documents(documents)

In [51]:
model_name = 'kakaobank/kf-deberta-base'
embed_model = HuggingFaceEmbedding(model_name=model_name, embed_batch_size=32)

config.json:   0%|          | 0.00/865 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/746M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/374 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.09M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.97M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [62]:
service_context = ServiceContext.from_defaults(node_parser=parser, embed_model=embed_model, llm=None)
# storage_context = StorageContext.from_defaults(persist_dir=persist_dir)

LLM is explicitly disabled. Using MockLLM.


In [257]:
len(documents)

834

In [258]:
# build index 
index = VectorStoreIndex.from_documents(
    documents, service_context=service_context, storage_context=feature_storage
) 

In [259]:
index.index_id

'1bfc29af-abdb-4905-87d1-61ab7cf4653c'

In [260]:
index.as_query_engine().query('렌탈 요금 청구').response

'Context information is below.\n---------------------\n임대인이 임대차계약이 있는 본인 소유의 주택을 담보로 대출을 받고, 대출이자는 임차인이 자동이체로 납부하는 주택담보대출대출신청자격- 국토교통부에서 정한 [집주인 담보대출 방식의 목돈 안드는 전세대출 이자부담부 특약]이포함된 임대차갱신계약을 체결한 임대인 중 아래 요건을 충족하는 고객님(신규임대차계약은 불가)ㅇ 임대차계약의 임차보증금이 수도권 기준 3억원(지방 2억원) 이하ㅇ 최초 또는 갱신임대차계약서가 부동산중개업소를 통한 계약서※ 수도권 : 서울특별시, 인천광역시,\n\n임차인에게 임대보증금을 반환하기 위한 자금을 임대인에게 지원하는 대출상품대출신청자격 임대차계약의 기간 만료(임대차계약 기간 만료 후 동일한 주택에서 동일한 임차인과 임대보증금을 인하하여 계약갱신하는 경우 포함) 등으로 임차인에게 임대보증금을 반환하기 위하여 대출을 받고자 하는 임대인으로 「주택금융신용보증서」가 발급되는 개인※ 임대인이 이미 임차인에게 임대보증금을 반환한 경우나 대상 주택 또는 대출신청인 및 배우자가 소유한 거주지의 부동산등기사항전부증명서에 소유권의 권리침해(압류, 가압류, 경매 등)가 있는 경우는 대출대상에서 제외됩니다.대출신청일 현재 임차인이 대상주택에 대항력(확정일자 및 전입)을 유지하고 있어야 합니다.※ 임차인이 임차권 등기 또는 전세권을 설정한 후 전출하였다면 전입 유지한 것으로 간주\n---------------------\nGiven the context information and not prior knowledge, answer the query.\nQuery: 렌탈 요금 청구\nAnswer: '

In [261]:
# set index_id to save multiple indexes to the same folder  -> persist 시 추가되는게 아니라 덮어 쓰여짐 .. => 해결 ! sotrage_context 지정안해줘서 그럼 
# storage context.from_defaults(persist_dir ~)은 docstore.json 파일이 있어야만 지정할 수 있는데 처음 인덱스 생성시에는 해당 부분 생략해서 docstore.json 파일 먼저 생성하기
index.set_index_id('scrapping_feature_chroma')

In [262]:
index.index_id

'scrapping_feature_chroma'

### Index data insert

In [135]:
data = pd.read_csv(os.path.join(data_path, 'financial', 'SHINHAN BANK_Financial_Product_Scraping_Result_20240104_LOAN_vec.csv'))
data.head(1)

,대출_고유키,금융상품명,상품특징,대출신청자격,상품설명
0,대출_0,혁신성장산업 지원자금,9대 선도산업 등 정부발표 혁신성장 분야에 대한 효율적 금융지원,○ (지원대상) 다음 중 하나에 해당하는 기업 혁신성장 정책금융협의회 선정 「혁신성...,9대 선도산업 등 정부발표 혁신성장 분야에 대한 효율적 금융지원


In [136]:
nona = data.copy() 
nona.dropna(subset='상품설명', inplace=True)
len(nona)

1343

In [137]:
category = 'loan'

id_list = nona.대출_고유키.values.tolist()
name_list = nona.금융상품명.values.tolist()
desc_list = nona.상품설명.values.tolist()

In [138]:
documents = []
for idx in range(len(desc_list)):
    doc = Document(text=desc_list[idx], doc_id=id_list[idx], metadata={"category": category, "name": name_list[idx]},
                   excluded_llm_metadata_keys = ['category', 'name'])
    documents.append(doc)

In [139]:
for doc in documents:
    index.insert(doc)

### Save Index 

In [263]:
index.storage_context.persist(persist_dir=os.path.join(index_path, 'feature'))

### Index Load 

In [49]:
vector_store = ChromaVectorStore(
    chroma_collection=qualification_collection,
)

In [66]:
service_context = ServiceContext.from_defaults(node_parser=parser, embed_model=embed_model, llm=None)

LLM is explicitly disabled. Using MockLLM.


In [67]:
index = VectorStoreIndex.from_vector_store(
    vector_store,
    service_context=service_context,
)

In [68]:
index.as_query_engine().query('렌탈 요금 청구').response

'Context information is below.\n---------------------\n- 국토교통부에서 정한 [집주인 담보대출 방식의 목돈 안드는 전세대출 이자부담부 특약]이포함된 임대차갱신계약을 체결한 임대인 중 아래 요건을 충족하는 고객님(신규임대차계약은 불가)ㅇ 임대차계약의 임차보증금이 수도권 기준 3억원(지방 2억원) 이하ㅇ 최초 또는 갱신임대차계약서가 부동산중개업소를 통한 계약서※ 수도권 : 서울특별시, 인천광역시,\n\n신용카드를 소지한 개인회원 중 신용도 및 이용실적에 의해 장기카드대출 대상자로 선정된 회원 (단, 대상자로 선정되어도 신청시점의 신용상태 변동에 따라 장기카드대출 신청이 제한될 수 있습니다.)계약기간3개월, 6개월, 9개월, 12개월, 18개월, 24개월, 36개월, 48개월, 60개월 중 선택가능 (고객님의 신용도, 이용실적 및 장기카드대출 신청금액에 따라 대출가능개월 수가 변동될 수 있으며, 거치기간 선택 시 거치기간 만큼 대출기간이 연장됩니다.)이자지급식주기1년을 365일(윤년은 366일)로 보고 1윌 단위로 계산유의사항신용정보관리대상 또는 연체 시 장기카드대출 이용이 제한될 수 있습니다.대출한도 및 이자율은 고객님의 신용도 및 이용실적에 따라 차등 적용됩니다.\n---------------------\nGiven the context information and not prior knowledge, answer the query.\nQuery: 렌탈 요금 청구\nAnswer: '

In [ ]:
from llama_index import StorageContext, load_index_from_storage, load_indices_from_storage

desc_storage_context = StorageContext.from_defaults(persist_dir=os.path.join(index_path, 'desc'))
features_storage_context = StorageContext.from_defaults(persist_dir=os.path.join(index_path, 'features'))
qualification_storage_context = StorageContext.from_defaults(persist_dir=os.path.join(index_path, 'qualification'))